# モデルの構築

In [1]:
import torch.nn as nn
import torch.nn.functional as F
#モデルの定義
class RegressionModel(nn.Module):
    def __init__(self,input_dim):
        super(RegressionModel,self).__init__()
        #入力層
        self.fc1 = nn.Linear(input_dim,32)
        #隠れ層
        self.fc2 = nn.Linear(32,128)
        self.dropout = nn.Dropout(p=0.2) #過学習防止のためのドロップアウト
        #出力層
        self.output = nn.Linear(128,1)
        #活性化関数
        self.relu = nn.ReLU()


    def forward(self,x):
        x = self.relu(self.fc1(x)) #入力層
        x = self.relu(self.fc2(x)) #隠れ層
        x = self.dropout(x) #ドロップアウト層
        x = self.output(x) #出力層
        return x

# データセットの読み込み

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

#学習時に使用する訓練データのパスの設定
train_path="../resources/house_prices/train_housing.csv"
data = pd.read_csv(train_path)

target_column="median_house_value"
x_train = data.drop(columns=[target_column])
y_train = data[target_column]
#訓練データの一部を検証用データにする
x_train,x_val,y_train,y_val = train_test_split(x_train,y_train,test_size=0.2,random_state=42)

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

#データをPyTorchテンソルに変換
x_train_tensor=torch.tensor(x_train.values,dtype=torch.float32)
y_train_tensor=torch.tensor(y_train.values,dtype=torch.float32).view(-1,1)
x_val_tensor=torch.tensor(x_val.values,dtype=torch.float32)
y_val_tensor=torch.tensor(y_val.values,dtype=torch.float32).view(-1,1)

#データローダの作成
batch_size=64
train_dataset = TensorDataset(x_train_tensor,y_train_tensor)
val_dataset = TensorDataset(x_val_tensor,y_val_tensor)
train_loader = DataLoader(train_dataset,batch_size=batch_size,shuffle=True)
val_loader = DataLoader(val_dataset,batch_size=batch_size)




## モデルの訓練と保存

In [3]:
#モデルのインスタンス化
input_dim = x_train.shape[1]
model = RegressionModel(input_dim)

#損失関数と最適化手法の設定
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(),lr=0.001)

import os
model_path = "../resources/models/best_model.pth"
best_val_loss = float('inf')
os.makedirs(os.path.dirname(model_path),exist_ok=True)
#学習のループ
num_epochs = 500
for epoch in range(num_epochs):
    model.train()
    running_loss=0.0
    for x_batch, y_batch in train_loader:
        #勾配の初期化
        optimizer.zero_grad()
        #順伝播
        outputs = model(x_batch)
        #損失の計算
        loss = criterion(outputs,y_batch)
        running_loss += loss.item()
        #逆伝播とパラメータの更新
        loss.backward()
        optimizer.step()
    #検証データでの評価
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for x_batch, y_batch in val_loader:
            #順伝播
            output = model(x_batch)
            #損失の計算
            loss = criterion(output,y_batch)
            val_loss += loss.item()
    print(f"Epoch [{epoch+1}/{num_epochs}], training loss: {running_loss/len(train_loader):.4f},\n validation loss:{val_loss/len(val_loader):.4f}")
    #検証時の損失が一番小さいモデルを保存
    if val_loss<best_val_loss:
        best_val_loss = val_loss
        script_model = torch.jit.script(model)
        script_model.save(model_path)
        print(f"TorchScriptモデルを'{model_path}'として保存しました。")

Epoch [1/500], training loss: 0.4211,
 validation loss:0.3161
TorchScriptモデルを'../resources/models/best_model.pth'として保存しました。
Epoch [2/500], training loss: 0.2994,
 validation loss:0.2941
TorchScriptモデルを'../resources/models/best_model.pth'として保存しました。
Epoch [3/500], training loss: 0.2781,
 validation loss:0.2792
TorchScriptモデルを'../resources/models/best_model.pth'として保存しました。
Epoch [4/500], training loss: 0.2670,
 validation loss:0.2723
TorchScriptモデルを'../resources/models/best_model.pth'として保存しました。
Epoch [5/500], training loss: 0.2588,
 validation loss:0.2655
TorchScriptモデルを'../resources/models/best_model.pth'として保存しました。
Epoch [6/500], training loss: 0.2526,
 validation loss:0.2620
TorchScriptモデルを'../resources/models/best_model.pth'として保存しました。
Epoch [7/500], training loss: 0.2510,
 validation loss:0.2602
TorchScriptモデルを'../resources/models/best_model.pth'として保存しました。
Epoch [8/500], training loss: 0.2437,
 validation loss:0.2643
Epoch [9/500], training loss: 0.2412,
 validation loss:0.2529
TorchScr

## 保存したモデルの推論結果

In [4]:
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset
#テスト用データの読み込み
test_data = pd.read_csv("./../resources/house_prices/test_housing.csv")
target_column="median_house_value"
x_test = test_data.drop(columns=[target_column])
y_test = test_data[target_column]

#データをPyTorchテンソルに変換
x_test_tensor=torch.tensor(x_test.values,dtype=torch.float32)
y_test_tensor=torch.tensor(y_test.values,dtype=torch.float32).view(-1,1)
#損失の設定
criterion = nn.MSELoss()

model = torch.jit.load(model_path)
model.eval()
with torch.no_grad():

    predict = model(x_test_tensor)
    test_loss = criterion(predict,y_test_tensor)
    print(f"Test loss(MSE): {test_loss.item():.4f}")

Test loss(MSE): 0.1966
